In [105]:
import importlib
import datetime as dt

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

from scipy import stats

import plotly.graph_objects as go
import plotly.express as px

import waldluft as wl

In [204]:
importlib.reload(wl)

<module 'waldluft' from '/Users/Janjo/Documents/MEINE DOKUMENTE/Projects/kaltluftstrom/waldluft.py'>

In [200]:
plt.style.use("janjo-v0-0.mplstyle")

In [221]:
jump = wl.Timed("raw_data/230409_jumpresponses", encoding="iso8859-1")

Successfully imported the following sensor data from raw_data/230409_jumpresponses:
    WTDL:
        W31 | W31-step.csv
        W32 | W32-step.csv
        W35.1| W35.1-step.csv
        W35.2| W35.2-step.csv
    SHT:


In [222]:
# correct times

ts = {}

corrs = {
    "W31": dt.datetime(2023,4,9,19,45,9),
    "W32": dt.datetime(2023,4,9,19,45,33),
    "W35.1": dt.datetime(2023,4,9,19,45,45),
    "W35.2": dt.datetime(2023,4,9,19,45,21),
}

for key, value in corrs.items():
    ts[key] = jump.timeseries[key].copy()
    ts[key].index = ts[key].index - value + dt.datetime(2023,4,9)

jump.timeseries = ts

In [223]:
fig = jump.plot_temp_time_interactive(
    plot_all=True,
    title="Sprungantwort WTDL1 in Eiswasser",
    xlabel="Zeitstempel (relativ)",
    ylabel="Temperatur / °C",
)
fig.update_xaxes(title_text='Zeitstempel (relativ)')
fig.update_yaxes(title_text='Temperatur / °C')
fig.show()

In [224]:
fig.add_annotation(
    x=dt.datetime(2023,4,9,0,5),
    y=10,
    text="tau Eiswasser: 75 s",
    xanchor="left",
    align="left",
    showarrow=False,
    font=dict(color="royalblue"),
)

fig.add_annotation(
    x=dt.datetime(2023,4,9,0,20),
    y=10,
    text="tau Luft: 540 s",
    xanchor="left",
    align="left",
    showarrow=False,
    font=dict(color="firebrick"),
)

In [225]:
# simulation

tau = 1.25 * 60 #seconds
start = 23.5 # °C
stop = 3.0 # °C

t = np.linspace(0.1,12*60,100)
t_ = [dt.datetime(2023,4,9) + dt.timedelta(seconds=seconds) for seconds in t]
y = (
    start
    + (
        (stop - start)
        * (1 - np.e ** (-(t / tau)))
    )
)

In [226]:
# simulation

tau2 = 9 * 60 #seconds
start2 = 3.0 # °C
stop2 = 21 # °C

t2 = np.linspace(0.1,28*60,100)
y2 = (
    start2
    + (
        (stop2 - start2)
        * (1 - np.e ** (-(t2 / tau2)))
    )
)
t2 = t2 + 11.5*60
t2_ = [dt.datetime(2023,4,9) + dt.timedelta(seconds=seconds) for seconds in t2]

In [227]:
fig = fig.add_trace(
    go.Scatter(
        x=t_,
        y=y,
        mode="lines",
        name="Simulation Wasser",
        line = dict(color='royalblue', width=3, dash='dash'),
        
    )
)
fig = fig.add_trace(
    go.Scatter(
        x=t2_,
        y=y2,
        mode="lines",
        name="Simulation Luft",
        line = dict(color='firebrick', width=3, dash='dash'),
        
    )
)

fig.show()

In [228]:
fig.write_html("html_graphics/jump_responses_icewater.html")